In [42]:
import time
import os
import copy
from glob import glob
import pandas as pd
import random
import numpy as np

In [43]:
# Cannot find  'XE19-010' and apoE_gt missing for XE10-045 and XE18-003
# 44 - 6 patients , 34 - 21 patients , 33 - 37 patients, 22-2 patients, 23-3 patients  

#take 6 patients from each 44, 33, 34

In [44]:
clinical_data = pd.read_csv("/mnt/new-nas/work/data/npsad_data/vivek/Metadata/clinical_metadata_v2_lates.csv")

In [45]:
clinical_data["ApoE_gt"].unique()

array([33., 34., 44., 23., nan, 24., 22.,  0.,  3.])

In [46]:
TRAIN_FOLDER = "/mnt/new-nas/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/train1"
TEST_FOLDER = "/mnt/new-nas/work/data/npsad_data/vivek/Datasets/amyb_wsi/test-patients/images"
csv_paths =  '/mnt/new-nas/work/data/npsad_data/vivek/csv_data/Internal_dataset_latest/'

train_xe_nums = os.listdir(TRAIN_FOLDER)
train_xe_nums.remove('.DS_Store')
train_xe_nums =  [os.path.join(TRAIN_FOLDER,x) for x in train_xe_nums]


test_xe_nums = os.listdir(TEST_FOLDER)
test_xe_nums.remove('.DS_Store')
test_xe_nums =  [os.path.join(TEST_FOLDER,x) for x in test_xe_nums]


train_xe_nums.extend(test_xe_nums)
train_test_xenum_pd = pd.DataFrame(train_xe_nums,columns=["WSI_filename"])

In [47]:
def create_quantify_df(train_df):
    train_full_quantify_df = pd.DataFrame()
    counter = 0
    no_csv_list =[]
    for files in train_df["WSI_filename"].unique():
        name = files.split("/")[-1]+"_quantify.csv"
        try: 
            quantify_df = pd.read_csv(os.path.join(csv_paths,name))
            quantify_df["Imaging_XENum"] = files.split("/")[-1].split("_")[0]
            if len(train_full_quantify_df)==0:
                train_full_quantify_df=quantify_df
            else:
                train_full_quantify_df=pd.concat([train_full_quantify_df,quantify_df])
        except:
            counter=counter+1
            no_csv_list.append(name)
            pass
    print("Not found csvs for :" , counter, no_csv_list)
    return train_full_quantify_df

In [48]:
all_detection_pd = create_quantify_df(train_test_xenum_pd)

Not found csvs for : 13 ['XE07-047_1_AmyB_1.json_quantify.csv', 'XE18-066_1_AmyB_1.json_quantify.csv', 'XE17-004_1_AmyB_1.json_quantify.csv', 'XE19-037_1_AmyB_1.json_quantify.csv', 'XE07-048_1_AmyB_1.json_quantify.csv', 'XE10-045_1_AmyB_1.json_quantify.csv', 'XE19-028_1_AmyB_1.json_quantify.csv', 'XE19-010_1_AmyB_1.json_quantify.csv', 'XE18-003_1_AmyB_1.json_quantify.csv', 'XE10-053_1_AmyB_1.json_quantify.csv', 'XE10-042_1_AmyB_1_quantify.csv', 'XE16-053_1_AmyB_1_quantify.csv', 'XE18-043_1_AmyB_1_quantify.csv']


In [49]:
all_detection_pd.columns

Index(['Unnamed: 0', 'image_name', 'label', 'confidence', 'brown_pixels',
       'core', 'neuritic', 'diffuse', 'caa', 'centroid', 'eccentricity',
       'area', 'equivalent_diameter', 'Imaging_XENum'],
      dtype='object')

In [50]:
train_test_xenum_pd = train_test_xenum_pd[~train_test_xenum_pd["WSI_filename"].isin([os.path.join(TEST_FOLDER, x) for x in ['XE10-042_1_AmyB_1', 'XE16-053_1_AmyB_1', 'XE18-043_1_AmyB_1']])]

In [51]:
train_test_xenum_pd

,WSI_filename
0,/mnt/new-nas/work/data/npsad_data/vivek/Datase...
1,/mnt/new-nas/work/data/npsad_data/vivek/Datase...
2,/mnt/new-nas/work/data/npsad_data/vivek/Datase...
3,/mnt/new-nas/work/data/npsad_data/vivek/Datase...
4,/mnt/new-nas/work/data/npsad_data/vivek/Datase...
...,...
74,/mnt/new-nas/work/data/npsad_data/vivek/Datase...
75,/mnt/new-nas/work/data/npsad_data/vivek/Datase...
76,/mnt/new-nas/work/data/npsad_data/vivek/Datase...
77,/mnt/new-nas/work/data/npsad_data/vivek/Datase...


In [52]:
train_test_xenum_pd["Imaging_XENum"] = train_test_xenum_pd["WSI_filename"].apply(lambda k:k.split("/")[-1].split("_")[0])

In [53]:
train_test_xenum_pd

,WSI_filename,Imaging_XENum
0,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE07-047
1,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE18-066
2,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE17-004
3,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE19-037
4,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE07-048
...,...,...
74,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE16-027
75,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE09-028
76,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE09-063
77,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE17-010


In [54]:
clinical_sel = clinical_data[clinical_data["Imaging_XENum"].isin(train_test_xenum_pd["Imaging_XENum"])][["Imaging_XENum","ApoE_gt"]]

In [55]:
train_test_xenum_pd = pd.merge(train_test_xenum_pd,clinical_sel,on="Imaging_XENum")

In [56]:
train_test_xenum_pd["ApoE_gt"].value_counts()

33.0    37
34.0    20
44.0     6
24.0     2
23.0     2
Name: ApoE_gt, dtype: int64

In [57]:
train_test_xenum_pd

,WSI_filename,Imaging_XENum,ApoE_gt
0,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE07-047,34.0
1,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE18-066,34.0
2,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE17-004,44.0
3,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE19-037,34.0
4,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE07-048,33.0
...,...,...,...
70,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE16-027,33.0
71,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE09-028,33.0
72,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE09-063,33.0
73,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE17-010,33.0


In [58]:
train_test_xenum_pd["data_source"] = train_test_xenum_pd["WSI_filename"].apply(lambda l: "train" if l.find(".json")!=-1 else "test")

In [59]:
train_test_xenum_pd["data_source"].value_counts()

test     66
train     9
Name: data_source, dtype: int64

In [60]:
#train_test_xenum_pd["image_count"] =  train_test_xenum_pd["WSI_filename"].apply(lambda l:len(glob(os.path.join(l,"images/*.png"))) if l.endswith("json") 
#                                                                                     else len(glob(os.path.join(l,"*.png")))  )

In [61]:
all_detection_pd.head(2)

,Unnamed: 0,image_name,label,confidence,brown_pixels,core,neuritic,diffuse,caa,centroid,eccentricity,area,equivalent_diameter,Imaging_XENum
0,0,x_5120_y_154624,Cored,0.976273,0,1,0,0,0,"(113.03329373615436, 145.34600443060327)",0.808533,31147,199.142140,XE08-017
1,1,x_38912_y_83968,Cored,0.922715,0,1,0,0,0,"(93.92957997020434, 187.96955951348588)",0.864045,44973,239.293548,XE08-017


In [62]:
def count_images(l):
    if len(all_detection_pd[all_detection_pd["Imaging_XENum"]==l])!=0:
        return len(all_detection_pd[all_detection_pd["Imaging_XENum"]==l])
    else:
        return len(glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/train1/",l+"_1_AmyB_1.json","images/*.png")))

train_test_xenum_pd["image_count"] =  train_test_xenum_pd["Imaging_XENum"].apply(lambda l:count_images(l))

In [63]:
train_test_xenum_pd["ApoE_gt"].unique()

array([34., 44., 33., nan, 24., 23.])

In [64]:
train_test_xenum_pd

,WSI_filename,Imaging_XENum,ApoE_gt,data_source,image_count
0,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE07-047,34.0,train,0
1,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE18-066,34.0,train,0
2,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE17-004,44.0,train,0
3,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE19-037,34.0,train,0
4,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE07-048,33.0,train,0
...,...,...,...,...,...
70,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE16-027,33.0,test,211
71,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE09-028,33.0,test,1103
72,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE09-063,33.0,test,229
73,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE17-010,33.0,test,1815


In [69]:
final_train_test_xenum_pd=pd.DataFrame()

for i in [34,44,33, 24]:
    tmp = train_test_xenum_pd[train_test_xenum_pd["ApoE_gt"]==i]
    random_list = random.sample(range(0, len(tmp)), len(tmp))
    tmp["random_ind"]=random_list
    tmp = tmp[tmp["random_ind"]<6]
    if len(final_train_test_xenum_pd)==0:
        final_train_test_xenum_pd = tmp
    else:
        final_train_test_xenum_pd=pd.concat([final_train_test_xenum_pd,tmp])

/tmp/ipykernel_376084/1900102874.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["random_ind"]=random_list


In [70]:
final_train_test_xenum_pd

,WSI_filename,Imaging_XENum,ApoE_gt,data_source,image_count,random_ind
6,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE19-028,34.0,train,0,2
20,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE15-022,34.0,test,671,3
26,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE16-023,34.0,test,77,1
54,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE16-014,34.0,test,383,4
56,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE07-056,34.0,test,1346,5
65,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE17-014,34.0,test,246,0
2,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE17-004,44.0,train,0,0
18,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE12-015,44.0,test,2342,4
49,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE13-005,44.0,test,457,2
58,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE09-041,44.0,test,36,5


In [71]:
final_train_test_xenum_pd.groupby(["ApoE_gt"])["image_count"].sum()

ApoE_gt
24.0    2912
33.0    5506
34.0    2723
44.0    4613
Name: image_count, dtype: int64

In [72]:
final_train_test_xenum_pd.groupby(["ApoE_gt"])["image_count"].sum()

ApoE_gt
24.0    2912
33.0    5506
34.0    2723
44.0    4613
Name: image_count, dtype: int64

In [73]:
final_train_test_xenum_pd.groupby(["random_ind","Imaging_XENum",])["image_count"].sum()

random_ind  Imaging_XENum
0           XE10-033         1172
            XE12-013          729
            XE17-004            0
            XE17-014          246
1           XE12-037          473
            XE13-029         2183
            XE14-047         1144
            XE16-023           77
2           XE13-005          457
            XE17-013         1540
            XE19-028            0
3           XE11-027          634
            XE14-004          127
            XE15-022          671
4           XE12-015         2342
            XE16-014          383
            XE17-030          607
5           XE07-056         1346
            XE09-041           36
            XE14-037         1587
Name: image_count, dtype: int64

In [74]:
final_train_test_xenum_pd["train_test_flag"] = np.where(final_train_test_xenum_pd["random_ind"].isin([3,4,1,0]),"Train","Val")

In [75]:
final_train_test_xenum_pd.groupby(["train_test_flag","ApoE_gt"])["image_count"].sum()

train_test_flag  ApoE_gt
Train            24.0       2912
                 33.0       2379
                 34.0       1377
                 44.0       4120
Val              33.0       3127
                 34.0       1346
                 44.0        493
Name: image_count, dtype: int64

In [80]:
final_train_test_xenum_pd.groupby(["train_test_flag","ApoE_gt"])["Imaging_XENum"].count()

train_test_flag  ApoE_gt
Train            24.0       2
                 33.0       4
                 34.0       4
                 44.0       4
Val              33.0       2
                 34.0       2
                 44.0       2
Name: Imaging_XENum, dtype: int64

In [81]:
final_train_test_xenum_pd.head(2)

,WSI_filename,Imaging_XENum,ApoE_gt,data_source,image_count,random_ind,train_test_flag
6,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE19-028,34.0,train,0,2,Val
20,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,XE15-022,34.0,test,671,3,Train


In [82]:
def map_label(class_name):
    if class_name == 24:
        return 1
    if class_name==33:
        return 0
    if class_name==34:
        return 1
    if class_name==44:
        return 1
    return -1

def get_image_paths(row):
    folder_path =  row["WSI_filename"]
    if folder_path.endswith("json"):
        files = glob(os.path.join(folder_path, "images/*.png"))
    else:
        files = glob(os.path.join(folder_path, "*.png"))
    temp_df =  pd.DataFrame({"crop_filepath":files})
    temp_df["WSI_filename"] = folder_path
    temp_df["class"] = row["ApoE_gt"]
    temp_df["Imaging_XENum"] = row["Imaging_XENum"]
    temp_df["random_ind"]=row["random_ind"]
    return temp_df

In [83]:
def create_image_data(train_test_flag):
    df = final_train_test_xenum_pd[final_train_test_xenum_pd["train_test_flag"]==train_test_flag]
    output_df = pd.DataFrame()
    for index, row in df.iterrows():
        tmp = get_image_paths(row)
        if len(output_df)==0:
            output_df=tmp
        else:
            output_df= pd.concat([output_df,tmp])
    output_df["apoe_label"]=output_df["class"].apply(lambda l:map_label(l))
    output_df["image_name"] = output_df["crop_filepath"].apply(lambda l:l.split("/")[-1].split(".")[0])
    output_df = pd.merge(output_df,all_detection_pd[["image_name","label","Imaging_XENum"]], on=["Imaging_XENum","image_name"],how="left") 
    return output_df

In [84]:
#final_train_test_xenum_pd["train_test_flag"]="Train"
train_df = create_image_data("Train")

In [85]:
train_df["label"].unique()

array([nan, 'Diffuse', 'Cored', 'CAA'], dtype=object)

In [86]:
train_df = train_df[~train_df["label"].isna()]

In [87]:
val_df = create_image_data("Val")
val_df = val_df[~val_df["label"].isna()]

In [89]:
train_df.to_csv("/mnt/new-nas/work/data/npsad_data/vivek/Datasets/amyb_wsi/apoe_training_dataset/train_detected_full2.csv")
val_df.to_csv("/mnt/new-nas/work/data/npsad_data/vivek/Datasets/amyb_wsi/apoe_training_dataset/val_detected_full2.csv")

In [90]:
train_df.groupby(["class","Imaging_XENum"])["image_name"].count()

class  Imaging_XENum
24.0   XE12-013          729
       XE13-029         2183
33.0   XE10-033         1172
       XE12-037          473
       XE14-004          127
       XE17-030          607
34.0   XE15-022          671
       XE16-014          383
       XE16-023           77
       XE17-014          246
44.0   XE11-027          634
       XE12-015         2342
       XE14-047         1144
Name: image_name, dtype: int64

In [91]:
train_df.groupby([ "class","random_ind"])["Imaging_XENum"].count()

class  random_ind
24.0   0              729
       1             2183
33.0   0             1172
       1              473
       3              127
       4              607
34.0   0              246
       1               77
       3              671
       4              383
44.0   1             1144
       3              634
       4             2342
Name: Imaging_XENum, dtype: int64

In [92]:
val_df = train_df[train_df["random_ind"].isin([3,2])]

In [93]:
val_df.groupby([ "class"])["Imaging_XENum"].count()

class
33.0    127
34.0    671
44.0    634
Name: Imaging_XENum, dtype: int64

In [94]:
val_df

,crop_filepath,WSI_filename,class,Imaging_XENum,random_ind,apoe_label,image_name,label
8,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE15-022,3,1,x_41984_y_126976,Diffuse
11,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE15-022,3,1,x_24576_y_136192,Diffuse
12,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE15-022,3,1,x_24576_y_136192,Diffuse
25,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE15-022,3,1,x_69632_y_75776,Cored
62,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE15-022,3,1,x_24576_y_106496,Diffuse
...,...,...,...,...,...,...,...,...
62357,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,33.0,XE14-004,3,0,x_47104_y_131072,Cored
62421,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,33.0,XE14-004,3,0,x_78848_y_104448,Cored
62534,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,33.0,XE14-004,3,0,x_32768_y_152576,Diffuse
62673,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,33.0,XE14-004,3,0,x_17408_y_60416,Cored


: 

In [187]:
train_df2 = train_df[~train_df["random_ind"].isin([3,2])]

In [188]:
train_df2.groupby([ "class"])["Imaging_XENum"].count()

class
33.0     698
34.0    2960
44.0    3943
Name: Imaging_XENum, dtype: int64

In [189]:
train_df2

,crop_filepath,WSI_filename,class,Imaging_XENum,random_ind,apoe_label,image_name,label
33,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE16-021,4,1,x_36864_y_119808,Cored
60,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE16-021,4,1,x_50176_y_88064,Cored
65,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE16-021,4,1,x_26624_y_136192,Cored
66,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE16-021,4,1,x_26624_y_136192,Cored
67,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE16-021,4,1,x_26624_y_136192,Cored
...,...,...,...,...,...,...,...,...
74288,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,33.0,XE10-046,4,0,x_45056_y_136192,Diffuse
74289,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,33.0,XE10-046,4,0,x_45056_y_136192,Diffuse
74309,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,33.0,XE10-046,4,0,x_45056_y_157696,Diffuse
74320,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,33.0,XE10-046,4,0,x_76800_y_165888,Diffuse


In [190]:
val_df.to_csv("/mnt/new-nas/work/data/npsad_data/vivek/Datasets/amyb_wsi/apoe_training_dataset/val_detected_full.csv")
train_df2.to_csv("/mnt/new-nas/work/data/npsad_data/vivek/Datasets/amyb_wsi/apoe_training_dataset/train_detected_full.csv")

In [191]:

#train_df = create_image_data("Train")

In [192]:
#val_df = create_image_data("Val")

In [193]:
train_df.to_csv("/mnt/new-nas/work/data/npsad_data/vivek/Datasets/amyb_wsi/apoe_training_dataset/train_full.csv")
val_df.to_csv("/mnt/new-nas/work/data/npsad_data/vivek/Datasets/amyb_wsi/apoe_training_dataset/val_full.csv")

In [194]:
train_df = pd.read_csv("/mnt/new-nas/work/data/npsad_data/vivek/Datasets/amyb_wsi/apoe_training_dataset/train_full.csv")
val_df = pd.read_csv("/mnt/new-nas/work/data/npsad_data/vivek/Datasets/amyb_wsi/apoe_training_dataset/val_full.csv")

In [195]:
train_df

,Unnamed: 0,crop_filepath,WSI_filename,class,Imaging_XENum,random_ind,apoe_label,image_name,label
0,33,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE16-021,4,1,x_36864_y_119808,Cored
1,60,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE16-021,4,1,x_50176_y_88064,Cored
2,65,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE16-021,4,1,x_26624_y_136192,Cored
3,66,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE16-021,4,1,x_26624_y_136192,Cored
4,67,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,34.0,XE16-021,4,1,x_26624_y_136192,Cored
...,...,...,...,...,...,...,...,...,...
7742,74288,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,33.0,XE10-046,4,0,x_45056_y_136192,Diffuse
7743,74289,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,33.0,XE10-046,4,0,x_45056_y_136192,Diffuse
7744,74309,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,33.0,XE10-046,4,0,x_45056_y_157696,Diffuse
7745,74320,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,/mnt/new-nas/work/data/npsad_data/vivek/Datase...,33.0,XE10-046,4,0,x_76800_y_165888,Diffuse


In [196]:
train_df["Imaging_XENum"].value_counts()

XE12-015    2342
XE16-021    1439
XE14-047    1144
XE15-007    1088
XE13-005     457
XE12-010     439
XE12-042     433
XE10-046     167
XE13-007     110
XE07-067      92
XE09-041      36
Name: Imaging_XENum, dtype: int64

In [197]:
len(train_df["WSI_filename"].unique())

11

In [198]:
def create_quantify_df(train_df):
    train_full_quantify_df = pd.DataFrame()
    counter = 0
    no_csv_list =[]
    for files in train_df["WSI_filename"].unique():
        name = files.split("/")[-1]+"_quantify.csv"
        try: 
            quantify_df = pd.read_csv(os.path.join(csv_paths,name))
            quantify_df["Imaging_XENum"] = files.split("/")[-1].split("_")[0]
            if len(full_quantify_df)==0:
                train_full_quantify_df=quantify_df
            else:
                train_full_quantify_df=pd.concat([train_full_quantify_df,quantify_df])
        except:
            counter=counter+1
            no_csv_list.append(name)
            pass
    print("Not found csvs for :" , counter, no_csv_list)
    return train_full_quantify_df

In [199]:
train_full_quantify_df = create_quantify_df(train_df)

Not found csvs for : 11 ['XE16-021_1_AmyB_1_quantify.csv', 'XE12-042_1_AmyB_1_quantify.csv', 'XE15-007_1_AmyB_1_quantify.csv', 'XE12-015_1_AmyB_1_quantify.csv', 'XE13-005_1_AmyB_1_quantify.csv', 'XE09-041_1_AmyB_1_quantify.csv', 'XE14-047_1_AmyB_1_quantify.csv', 'XE07-067_1_AmyB_1_quantify.csv', 'XE13-007_1_AmyB_1_quantify.csv', 'XE12-010_1_AmyB_1_quantify.csv', 'XE10-046_1_AmyB_1_quantify.csv']


In [200]:
val_full_quantify_df = create_quantify_df(val_df)

Not found csvs for : 2 ['XE09-041_1_AmyB_1_quantify.csv', 'XE13-007_1_AmyB_1_quantify.csv']


In [201]:
len(full_quantify_df["Imaging_XENum"].unique())

NameError: name 'full_quantify_df' is not defined

In [ ]:
def flag_images_with_detection(row):
    os.path.join()